# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 165b887f


In [5]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [15]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [16]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [18]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_3", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
🔄 [todo_3] Research sleep improvement strategies (in_progress)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [19]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/adrianbarcan/Projects/AIBootcamp/AIE9/07_Deep_Agents/workspace


In [20]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
(empty directory)


In [21]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [22]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [23]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/adrianbarcan/Projects/AIBootcamp/AIE9/07_Deep_Agents/workspace


In [24]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I've successfully created your personalized sleep improvement plan. Here's what I've accomplished:

## 🌙 Your Complete Sleep Improvement Plan

I've saved a comprehensive, personalized sleep improvement plan to **`/your_personalized_sleep_plan.md`** that specifically addresses your three main challenges:

### 🎯 **Plan Highlights:**

**For Your Inconsistent Bedtimes (10pm-1am):**
- Structured 4-week approach to establish consistent sleep schedule
- Specific recommendation: 10:30-11:00 PM bedtime with 7:00 AM wake time
- Gradual implementation to avoid overwhelming changes

**For Your Phone Usage in Bed:**
- Immediate action plan: move phone charger out of bedroom tonight
- Blue light filter installation and "digital sunset" at 10:00 PM
- Phone-free alternatives like physical books and sleep meditations

**For Your Morning Fatigue:**
- Morning hydration protocol (16oz water immediately upon waking)
- Bright light exposure routine within first hour
- Sleep cycle op

In [25]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep hygiene practices (completed)
✅ [todo_5] Create personalized sleep schedule recommendations (completed)
✅ [todo_7] Develop phone/screen time management strategy (completed)
✅ [todo_9] Design morning routine to combat fatigue (completed)
✅ [todo_11] Create implementation timeline (completed)
✅ [todo_13] Save complete sleep improvement plan to file (completed)
✅ [todo_8] Research sleep schedule consistency and bedtime routines (completed)
✅ [todo_10] Investigate screen time and phone usage impact on sleep (completed)
✅ [todo_12] Analyze morning fatigue causes and solutions (completed)
✅ [todo_14] Research progressive implementation approaches (completed)
✅ [todo_16] Investigate sleep environment optimization (completed)
✅ [todo_18] Research behavioral and lifestyle factors (completed)
✅ [todo_20] Compile comprehensive research summary (completed)


Workspace contents:
  [FIL

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
When might explicit planning overhead slow things down?
- When having straightforward queries like "What's the recommended daily water intake?", creating todos adds unnecessary overhead. The agent spends time creating, updating, and tracking tasks when it could just answer the question directly
- In real-time conversations where users expect quick responses, the planning cycle introduces latency, making the conversation very long and making users to lose interest
- When requirements change rapidly during task execution, maintaining an accurate todo list can become time consuming and not add any real benefits

How granular should todo items be?
- Todos should be actionable and can be implemented idenpendently, typically 1-3 tool calls each
- If a todo requires multiple sub-decisions, it's too complex and needs breaking down. If completing it takes one obvious action, it's too granular (For example: "Create wellness plan" is good but "Open sleep_notes.md" is too granular)

What happens if the agent creates todos but never completes them?
- Pending todos accumulate, cluttering the agent's context and creating confusion about what task are needed
- Users see an incomplete task list with no clear explanation of why items remain open
- Planning effort is spent but never 

As a good practice, we should be setting a maximum number of todos and making the agent review and clean up stale todos periodically 

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
What goes in files vs. in the prompt?
- Large health document (16KB) should go into the File system because it is too large for the prompt. We can read relevant sections on-demand
- Historical user metrics should go into the File system because metrics accumulate over time. It can be stored in a structured file (JSON or CSV) and queries as needed
- Allergies & medications should be kept in prompt because these needs to be present in every response
- Current conversation context should also be kept in prompt because it is relevant in the conversation

What should never be offloaded?
- The allergies, medications or other conditions should be always present in the prompt because if we keep them in a file, the agent might forget to read that file and recommend someting dangerous. As a golden rule, if it's something that can cause harm if ignored, it should stay in the prompt. Everything else can be offloaded to files or memory and retrieved on-demand to keep the context lean and focused.

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [26]:
### YOUR CODE HERE ###

# Step 1: Reset todo store for fresh demo
TODO_STORE.clear()
# Step 2: Add a tool to read from the data folder
@tool
def read_wellness_guide() -> str:
    """Read the HealthWellnessGuide.txt from the data folder.
    
    Returns:
        Contents of the wellness guide document
    """
    guide_path = Path("data/HealthWellnessGuide.txt")
    if not guide_path.exists():
        return "Error: HealthWellnessGuide.txt not found in data folder"
    return guide_path.read_text()

# Step 3: Configure filesystem backend for research output
research_workspace = Path("workspace").absolute()
research_backend = FilesystemBackend(
    root_dir=str(research_workspace),
    virtual_mode=True
)

# Step 4: Define research tools (todo tools + wellness guide reader)
research_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_wellness_guide,
]

# Step 5: Create the research agent with a research-focused system prompt
research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=research_tools,
    backend=research_backend,
    system_prompt="""You are a Wellness Research Agent that specializes in analyzing health information and creating structured reports.
        When given a research task:
        1. First, create a detailed todo list to track your research process
        2. Read the HealthWellnessGuide using the read_wellness_guide tool
        3. Extract and analyze relevant information for the topic
        4. Update todo status as you complete each step
        5. Save your findings to a structured markdown file in the workspace
        6. Provide a summary of your research when complete
        For your reports:
        - Use clear headings and subheadings
        - Include evidence-based strategies with explanations
        - Cite information from the wellness guide where applicable
        - Make recommendations actionable and practical
        Be thorough, evidence-based, and well-organized."""
)
print("Research Agent created!")
print(f"Output will be saved to: {research_workspace}")

# Step 6: Test with the stress management research task
result = research_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]
})
print("\n" + "="*50)
print("RESEARCH AGENT RESPONSE:")
print("="*50)
print(result["messages"][-1].content)

# Step 7: Show the current todo status
print("\n" + "="*50)
print("FINAL TODO STATUS:")
print("="*50)
print(list_todos.invoke({}))

# Step 8: List files created in workspace
print("\n" + "="*50)
print("FILES CREATED:")
print("="*50)
print(ls.invoke({"path": "."}))


Research Agent created!
Output will be saved to: /Users/adrianbarcan/Projects/AIBootcamp/AIE9/07_Deep_Agents/workspace

RESEARCH AGENT RESPONSE:
## Research Complete: Comprehensive Stress Management Guide

I have successfully completed your research on stress management techniques and created a comprehensive guide with evidence-based strategies. Here's a summary of what I've accomplished:

### Key Findings from the Wellness Guide Research:

**6 Evidence-Based Stress Management Strategies Identified:**

1. **Deep Breathing Techniques** - Box breathing (4-4-4-4 method) that activates the parasympathetic nervous system
2. **Progressive Muscle Relaxation** - Systematic tension and release to break the stress-tension cycle
3. **Mindfulness Meditation** - Present-moment awareness practices proven to reduce cortisol and improve emotional regulation
4. **Physical Exercise and Movement** - Both immediate relief exercises and long-term fitness strategies
5. **Cognitive Reframing** - CBT-based te

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [27]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [29]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [30]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I accomplished:

## ✅ Project Complete!

### What You Now Have:
📄 **Complete Guide Saved**: `/comprehensive-morning-routine-guide.md` - A 50+ section comprehensive guide

### Key Features of Your Guide:

🧠 **Science-Based Foundation**
- Circadian rhythm optimization
- Cortisol patterns and energy cycles
- Neuroplasticity and habit formation
- Research-backed explanations for why morning routines work

💪 **Comprehensive Practical Sections**
- **Exercise**: Options from 5-60 minutes, indoor/outdoor, all fitness levels
- **Nutrition**: Optimal breakfast timing, sustained energy foods, hydration strategies
- **Mindset**: Meditation, journaling, visualization, stress-reduction techniques

🎯 **Actionable Implementation**
- 4 different routine templates (20-90 minutes)
- Step-by-step building process
- Challenge solutions and troubleshooting
- Advanced optimization 

In [31]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines and energy (completed)
✅ [todo_3] Research practical morning routine elements (completed)
✅ [todo_5] Create comprehensive morning routine guide (completed)
✅ [todo_7] Save guide as formatted markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive-morning-routine-guide.md (13488 bytes)
  [FILE] comprehensive_morning_routine_guide.md (13488 bytes)
  [FILE] exercise_morning_energy.txt (1427 bytes)
  [FILE] mindset_morning_energy.txt (801 bytes)
  [FILE] morning_energy_guide.md (16867 bytes)
  [FILE] nutrition_morning_energy.txt (1098 bytes)
  [FILE] personalized_sleep_improvement_plan.md (9480 bytes)
  [DIR] research/
  [FILE] sleep_hygiene_research.md (14395 bytes)
  [FILE] sleep_research_executive_summary.md (8682 bytes)
  [FILE] stress_management_guide.md (12230 bytes)
  [FILE] your_personalized_sleep_plan.md (6564 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [32]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_adrian"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Adrian"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_adrian
  name: {'value': 'Adrian'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [33]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [34]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [35]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_adrian. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Adrian! Great to connect with you again. Based on your profile, I can see your primary goal is to improve energy levels and your secondary goal is better sleep. I also note you prefer morning exercise and have mild anxiety, so I'll tailor my recommendations accordingly.

Here's a personalized exercise routine designed specifically for your goals:

## **Morning Energy-Boosting Routine (30-40 minutes)**

### **Phase 1: Gentle Wake-Up (5-8 minutes)**
- **Light stretching** - Focus on neck, shoulders, and spine to release overnight tension
- **Deep breathing exercises** - 2-3 minutes of box breathing (4-4-4-4 count) to calm anxiety and center yourself
- **Gentle joint mobility** - Arm circles, hip circles, ankle rolls

### **Phase 2: Cardio for Energy (15-20 minutes)**
- **Brisk walking or light jogging** - Outdoors if possible for natural light exposure (helps with sleep regulation)
- **Alternative:** Indoor dance, jumping jacks, or bodyweight circuits
- **Intensity:** 

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [36]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [37]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [38]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [39]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [40]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Wellness Recommendations

Based on your assessment, here are my specific recommendations organized by timeline:

### 🚀 **Immediate Action Items (Start Today):**

1. **Set a consistent sleep schedule** - Choose a bedtime and wake time that allows for 7-8 hours of sleep, even on weekends
2. **Implement desk breaks** - Set a timer for every 30-45 minutes to stand, stretch, or walk for 2-3 minutes
3. **Create a wind-down routine** - No screens 1 hour before bed, dim lights, and try relaxation techniques

### 📋 **Short-term Goals (1-2 weeks):**

1. **Start a walking routine** - Begin with 15-20 minute walks after work or during lunch breaks
2. **Optimize your evening meals** - Focus on protein-rich vegetarian dinners 3 hours before bedtime
3. **Track your progress** - Monitor sleep quality, daily steps, and energy levels

### 🎯 **Long-term Goals (1-3 months):**

1. **Establish regular exercise routine** - Build up to 150 minutes of moderate activity per 

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [41]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.18


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [42]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [43]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [44]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
## 🎉 Your Complete 2-Week Wellness Program is Ready!

Alex, I'm excited to present your comprehensive wellness program! Working with my specialist team, I've created a fully integrated 2-week plan that addresses all your goals:

### ✅ **What You Now Have:**

**4 Complete Reference Files:**
1. **Master Wellness Program** (`/wellness_plans/alex_master_wellness_program.txt`) - Your integrated daily guide
2. **Exercise Plan** (`/wellness_plans/alex_exercise_plan.txt`) - 3x/week, 30-minute progressive workouts
3. **Vegetarian Meal Plan** (`/wellness_plans/alex_meal_plan.txt`) - 2 weeks of meals, recipes, and shopping lists
4. **Stress & Sleep Guide** (`/wellness_plans/alex_stress_sleep_plan.txt`) - Daily techniques for work stress and better sleep

### 🌟 **Key Features of Your Program:**

**🏋️ Exercise (3x/week, 30 minutes)**
- Progressive 2-week strength and cardio plan
- No equipment needed - bodyweight focus
- Modifications for all fitness levels
- Built for cons

In [45]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create 2-week exercise routine plan (completed)
✅ [todo_3] Design vegetarian meal plan (completed)
✅ [todo_5] Develop stress management and sleep optimization plan (completed)
✅ [todo_7] Integrate all components into master wellness program (completed)
✅ [todo_9] Create reference files for each component (completed)

GENERATED FILES
  [FILE] comprehensive-morning-routine-guide.md (13488 bytes)
  [FILE] comprehensive_morning_routine_guide.md (13488 bytes)
  [FILE] exercise_morning_energy.txt (1427 bytes)
  [DIR] exercise_routines/
  [FILE] mindset_morning_energy.txt (801 bytes)
  [FILE] morning_energy_guide.md (16867 bytes)
  [FILE] nutrition_morning_energy.txt (1098 bytes)
  [FILE] personalized_sleep_improvement_plan.md (9480 bytes)
  [DIR] research/
  [FILE] sleep_hygiene_research.md (14395 bytes)
  [FILE] sleep_research_executive_summary.md (8682 bytes)
  [FILE] stress_management_guide.md (12230 bytes)
  [FILE] stress_management_sleep_optimization_plan_us

In [46]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of sleep_research_executive_summary.md:
# Executive Summary: Evidence-Based Sleep Improvement Research

## Overview
This research synthesizes current scientific literature on sleep hygiene to provide actionable, evidence-based recommendations for improving sleep quality. The focus areas align with the most common sleep challenges: inconsistent bedtimes, excessive screen time, and morning fatigue.

## Key Research Findings

### 1. Sleep Schedule Consistency
**Scientific Foundation**: The circadian rhythm, controlled by the suprachiasmatic nucleus, requires consistent timing to function optimally.

**Critical Insights**:
- Consistent wake time is MORE important than consistent bedtime initially
- "Social jet lag" from weekend schedule changes can persist for days
- Gradual adjustments (15-30 minutes every 2-3 days) are more sustainable than dramatic changes
- Individual chronotypes vary, but consistency benefits everyone

### 2. Screen Time Impact & Solutions
**Scientific Mecha

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
When should subagents share tools vs have distinct tools?
Subagents should share read tools, but restrict write tools to the subagent responsible for that type of task. For example research and writing subagents both need read_file. However, there should be distinct tools when specialization requires unique capabilities (Nutrition subagent gets calculate_calories, Exercise subagent gets track_workout)

How do you decide which model to use for each subagent?
You can decide on the model by taking into account:
- Cost: use cheaper models where quality permits (simple extraction such as parsing or formatting, simple decisions such as routing)
- Latency: Faster models for user-facing or frequently-called subagents
- Accuracy requirements: High-stakes tasks requires better models

What's the right granularity for subagent specialization?
- Each subagent should have one clear domain with enough scope to justify the spawning overhead, but not so broad that it loses focus
- A good subagent can be described in one sentence and needs 3-7 tools to do its job well:
    - Too narrow: "Subagent that calculates breakfast calories" — this should just be a tool
    - Too broad: "Subagent that handles all wellness" — this defeats the purpose of specialization
    - Good: "Nutrition Advisor that handles meal planning, calorie tracking, and dietary analysis"

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
Moving from a prototype or demo app to a **production wellness application** using Deep Agents, we need:
1. Safety Guardrails for Health Advice
Health advice is high-stakes. You need:

- Medical disclaimer — Responses should include "consult a healthcare professional"
- Blocked recommendations — Never suggest stopping medications, specific dosages, or diagnose conditions
- Allergy/medication checks — Cross-reference all food/supplement suggestions against user's safety profile
- Escalation triggers — If user mentions chest pain, suicidal thoughts, etc., immediately suggest professional help
- Output validation — Review agent responses before delivery using a safety classifier

2. Persistent Storage (Not In-Memory)
The notebook demos use TODO_STORE = {} which disappears on restart. Production needs:

- Database for user data — PostgreSQL or MongoDB for profiles, metrics, preferences
- File storage backend — S3 or similar for research notes, plans, and generated documents
- LangGraph Store — For long-term memory across sessions (already designed for this)
- Checkpointing — Save agent state so users can resume interrupted tasks

3. Multi-User Support and Isolation
- User namespacing — Each user gets their own Store namespace, file directory, and todo list
- Authentication — Verify user identity before accessing their data
- Data isolation — Subagents should only access the current user's context, never cross-user data
- Rate limiting per user — Prevent any single user from exhausting resources

4. Monitoring and Observability
- LangSmith tracing — Track every agent step, tool call, and subagent spawn
- Latency monitoring — Alert if responses take too long
- Error tracking — Log failures with full context for debugging
- Usage dashboards — See which features are used, where users get stuck
- Safety incident logging — Flag and review any triggered guardrails

5. Cost Management with Subagents
Subagents multiply API costs quickly. Strategies:

- Model tiering — Use cheaper models (Haiku/GPT-4o-mini) for simple subagents
- Subagent budgets — Limit max tokens per subagent invocation
- Caching — Cache common subagent responses (e.g., generic exercise advice)
- Spawn limits — Cap how many subagents can run per request
- Cost attribution — Track spend per user, per subagent type, per feature


---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [48]:
### YOUR CODE HERE ###

# Step 1: Reset stores for fresh demo
TODO_STORE.clear()

# Initialize memory store for cross-session memory
from langgraph.store.memory import InMemoryStore
wellness_memory_store = InMemoryStore()

# Step 2: Define memory tools for user preferences
@tool
def get_user_preferences(user_id: str) -> str:
    """Retrieve user's wellness preferences and challenge settings from memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User preferences as formatted text
    """
    namespace = (user_id, "wellness_challenge")
    items = list(wellness_memory_store.search(namespace))
    
    if not items:
        return f"No preferences found for {user_id}. This appears to be a new user."
    
    result = [f"Wellness Challenge Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preferences(user_id: str, key: str, value: str) -> str:
    """Save user's wellness preferences to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key (e.g., 'focus_area', 'difficulty', 'wake_time')
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "wellness_challenge")
    wellness_memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}={value}' for {user_id}"

@tool
def log_daily_checkin(user_id: str, day: int, notes: str, mood: str, completed: bool) -> str:
    """Log a daily check-in for the 30-day challenge.
    
    Args:
        user_id: The user's unique identifier
        day: Day number of the challenge (1-30)
        notes: User's notes about the day
        mood: User's mood rating (e.g., 'great', 'good', 'okay', 'struggling')
        completed: Whether the daily challenge was completed
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "daily_checkins")
    checkin_data = {
        "day": day,
        "notes": notes,
        "mood": mood,
        "completed": completed
    }
    wellness_memory_store.put(namespace, f"day_{day}", checkin_data)
    
    status = "✅ Completed" if completed else "⬜ Incomplete"
    return f"Logged Day {day} check-in: {status}, Mood: {mood}"

@tool
def get_challenge_progress(user_id: str) -> str:
    """Get the user's progress through the 30-day challenge.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        Summary of challenge progress
    """
    namespace = (user_id, "daily_checkins")
    items = list(wellness_memory_store.search(namespace))
    
    if not items:
        return f"No check-ins found for {user_id}. Challenge may not have started."
    
    total_days = len(items)
    completed_days = sum(1 for item in items if item.value.get("completed", False))
    moods = [item.value.get("mood", "unknown") for item in items]
    
    result = [
        f"Challenge Progress for {user_id}:",
        f"  Days logged: {total_days}/30",
        f"  Days completed: {completed_days}/{total_days}",
        f"  Completion rate: {(completed_days/total_days*100):.0f}%" if total_days > 0 else "  Completion rate: N/A",
        f"  Recent moods: {', '.join(moods[-5:])}",
        "",
        "Recent check-ins:"
    ]
    
    for item in sorted(items, key=lambda x: x.value.get("day", 0))[-5:]:
        v = item.value
        status = "✅" if v.get("completed") else "⬜"
        result.append(f"  Day {v.get('day')}: {status} | {v.get('mood')} | {v.get('notes', 'No notes')[:50]}")
    
    return "\n".join(result)

# Step 3: Define subagent configurations

# Nutrition subagent - specializes in meal planning and dietary advice
nutrition_subagent = {
    "name": "nutrition-coach",
    "description": "Use this agent for meal planning, dietary advice, and nutrition-related questions. It creates personalized meal suggestions based on user goals.",
    "system_prompt": """You are a Nutrition Coach specializing in wellness challenges. Your role is to:
1. Create practical, healthy meal suggestions
2. Provide nutrition tips aligned with energy, sleep, or fitness goals
3. Adapt recommendations based on dietary preferences
4. Focus on sustainable eating habits, not extreme diets

Keep advice practical and achievable. Suggest whole foods over supplements when possible.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",  # Cheaper model for routine nutrition advice
}

# Fitness subagent - specializes in exercise planning
fitness_subagent = {
    "name": "fitness-coach",
    "description": "Use this agent for exercise routines, workout planning, and physical activity guidance. It creates personalized fitness plans based on user goals and fitness level.",
    "system_prompt": """You are a Fitness Coach specializing in wellness challenges. Your role is to:
1. Design achievable workout plans for all fitness levels
2. Suggest exercises that fit into busy schedules
3. Provide form tips and injury prevention advice
4. Adapt routines based on user feedback and progress

Focus on consistency over intensity. A 10-minute workout done daily beats an hour workout done rarely.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

# Step 4: Configure filesystem backend for context management
challenge_workspace = Path("workspace").absolute()
challenge_backend = FilesystemBackend(
    root_dir=str(challenge_workspace),
    virtual_mode=True
)

# Step 5: Combine all tools for the main coordinator
wellness_coach_tools = [
    # Planning tools
    write_todos,
    update_todo,
    list_todos,
    # Memory tools
    get_user_preferences,
    save_user_preferences,
    log_daily_checkin,
    get_challenge_progress,
    # Data access
    read_wellness_guide,
]

# Step 6: Create the main Wellness Coach agent
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=wellness_coach_tools,
    backend=challenge_backend,
    subagents=[nutrition_subagent, fitness_subagent],
    system_prompt="""You are a 30-Day Wellness Challenge Coach that helps users transform their health habits.

## Your Capabilities:
- **Planning**: Use todos to track the user's 30-day journey
- **Memory**: Remember user preferences, goals, and progress across sessions
- **Subagents**: Delegate to nutrition-coach and fitness-coach for specialized advice
- **Files**: Save challenge plans, daily notes, and weekly summaries to the workspace

## For New Users:
1. Ask about their wellness goals (energy, sleep, fitness, stress, weight)
2. Assess their current habits and lifestyle constraints
3. Save their preferences to memory
4. Generate a personalized 30-day plan with daily micro-challenges
5. Save the plan as a markdown file

## For Returning Users:
1. Load their preferences and check progress
2. Log their daily check-in (mood, completion, notes)
3. Adapt today's recommendations based on recent feedback
4. At week ends (Day 7, 14, 21, 30), create and save a weekly summary report

## Key Principles:
- Start small: Day 1 challenges should take <5 minutes
- Build gradually: Increase difficulty as habits form
- Celebrate wins: Acknowledge every completed day
- Adapt: If user is struggling, reduce intensity rather than letting them quit

Be encouraging, practical, and personalized. Reference their preferences and history to show you remember them."""
)

print("30-Day Wellness Challenge Coach created!")
print(f"Files will be saved to: {challenge_workspace}")

# Step 7: Test - Create a new user's 30-day challenge
print("\n" + "="*60)
print("TEST 1: Creating a new user's 30-day challenge")
print("="*60)

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! I'm user_adrian and I want to start a 30-day wellness challenge. 
        
My goals:
- Improve my energy levels (I feel tired by 3pm every day)
- Sleep better (I have trouble falling asleep)
- Start exercising (currently sedentary, very beginner)

Constraints:
- I work from the office
- Only have 15-20 minutes per day for exercise in the morning
- I have a dog that needs to be walked every day

Please create my personalized 30-day plan!"""
    }]
})

print("\nCoach Response:")
print(result["messages"][-1].content)

# Step 8: Show todo status and files created
print("\n" + "="*60)
print("TODO STATUS:")
print("="*60)
print(list_todos.invoke({}))

print("\n" + "="*60)
print("FILES CREATED:")
print("="*60)
print(ls.invoke({"path": "."}))

# Step 9: Test - Simulate a daily check-in
print("\n" + "="*60)
print("TEST 2: Day 3 check-in and adaptation")
print("="*60)

checkin_result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi, it's user_adrian checking in for Day 3!

Yesterday (Day 2) went well - I did the morning stretch routine.
Today I'm feeling a bit low energy and stressed because of work deadlines.

Can you log my check-in and give me today's challenge? Maybe something that helps with stress?"""
    }]
})

print("\nCoach Response:")
print(checkin_result["messages"][-1].content)

# Step 10: Check memory state
print("\n" + "="*60)
print("USER MEMORY STATE:")
print("="*60)
print(get_user_preferences.invoke({"user_id": "user_adrian"}))
print("\n")
print(get_challenge_progress.invoke({"user_id": "user_adrian"}))

# Step 11: Test - Simulate Week 1 completion (Day 7)
print("\n" + "="*60)
print("TEST 3: Day 7 - End of Week 1 Summary")
print("="*60)
# First, simulate some check-ins for days 4-6 to have data for the summary
log_daily_checkin.invoke({"user_id": "user_adrian", "day": 4, "notes": "Morning stretch done, felt energized", "mood": "good", "completed": True})
log_daily_checkin.invoke({"user_id": "user_adrian", "day": 5, "notes": "Tried 5-min meditation, hard to focus", "mood": "okay", "completed": True})
log_daily_checkin.invoke({"user_id": "user_adrian", "day": 6, "notes": "Tried running for 10 minutes, loved it!", "mood": "great", "completed": True})
print("Simulated check-ins for Days 4-6")
print(get_challenge_progress.invoke({"user_id": "user_adrian"}))
# Now trigger the Day 7 summary
week1_result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi, it's user_adrian! Today is Day 7 - I completed my first week!
I did the morning walk today and I'm feeling accomplished.
Please log my Day 7 check-in and create my Week 1 summary report!"""
    }]
})
print("\nCoach Response:")
print(week1_result["messages"][-1].content)
# Show the weekly summary file was created
print("\n" + "="*60)
print("FILES AFTER WEEK 1:")
print("="*60)
print(ls.invoke({"path": "."}))
# Optionally read the weekly summary
print("\n" + "="*60)
print("WEEK 1 SUMMARY CONTENT (if created):")
print("="*60)
try:
    # Try to find and read the weekly summary file
    for f in challenge_workspace.iterdir():
        if "week" in f.name.lower() and "summary" in f.name.lower():
            print(f.read_text()[:2000] + "..." if len(f.read_text()) > 2000 else f.read_text())
            break
    else:
        print("Weekly summary file not found - check ls output for actual filename")
except Exception as e:
    print(f"Could not read summary: {e}")

30-Day Wellness Challenge Coach created!
Files will be saved to: /Users/adrianbarcan/Projects/AIBootcamp/AIE9/07_Deep_Agents/workspace

TEST 1: Creating a new user's 30-day challenge

Coach Response:
## 🎉 Your Personalized 30-Day Wellness Challenge is Ready!

Adrian, I've created a comprehensive plan tailored specifically to your goals and lifestyle. Here's what makes your plan special:

### 🎯 **Designed for YOUR Life:**
- **15-20 minute morning workouts** that fit your schedule
- **Office-friendly strategies** for sustained energy 
- **Dog walk integration** to maximize your existing commitment
- **Gradual progression** from complete beginner to confident exerciser

### 🏋️ **Your Fitness Journey:**
- **Week 1:** Gentle 15-minute routines to build the habit
- **Week 2:** 15-20 minutes with increased intensity 
- **Week 3:** Full 20-minute endurance building
- **Week 4:** Advanced movements and habit mastery

### 🥗 **Energy & Sleep Solutions:**
- **Morning fuel:** Protein-rich breakfast

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)